# LIBRARIES & FUNCTIONS

In [1]:
# LIBRARIES #

import numpy as np

from Gafchromic import Radiochromic_RB

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect, Cross
from bokeh.layouts import row, gridplot, column

import SimpleITK as sitk 

import pydicom



output_notebook()

Loading BokehJS ...

In [2]:
# FONCTIONS #



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleRGBArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor,:]\
                        .reshape((newsizey, subfactor, newsizex, subfactor, 3)).mean(3).mean(1)
    
    return newarray



# reading dose matrix in axial plane at isocenter 
#    from dose and plan dicom files : #
# @params:
#  doseFilename : filename of the dicom file containing the dose matrix
#  planFilename : filename of the dicom file containing the RT plan
#  verbose : writes the print output ?
def readDoseMatrix_AxialPlane_Isocenter(doseFilename, planFilename, offset=0, verbose=False):

    ds_dose = pydicom.read_file(doseFilename)
    ds_plan = pydicom.read_file(planFilename)
    
    if verbose: print('Dose files read!')

    
    # Isocenter Coordinates from plan file:
    iso_x=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[0]
    iso_y=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[1]
    iso_z=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[2]

    if verbose: print ('  > isocenter coordinates = (', iso_x, ";", iso_y, ";", iso_z, ")")


    # Dose image coordinates from dose file:
    position_image=list(ds_dose['0020','0032'].value) # +x left, +y post, +z head   
    position_image_x=list(ds_dose['0020','0032'].value)[0]
    position_image_y=list(ds_dose['0020','0032'].value)[1]
    position_image_z=list(ds_dose['0020','0032'].value)[2]

    if verbose: print ('  > Dose image position = (', position_image_x, ";", position_image_y, ";", position_image_z, ")")


    # Dose matrix in axial plane at isocenter:
    dim_dose= ds_dose.pixel_array.shape # disposition y, z, x?
    
    if verbose: print ('  > dose matrix size = ', dim_dose)

    pixel_spacing_x=(ds_dose['0028','0030'].value[0]) #definition de la résolution de la matrice de dose
    pixel_spacing_z=(ds_dose['0028','0030'].value[1])
    pixel_spacing_y=(ds_dose['3004','000C'].value[1])
    
    if verbose: print ('  > pixel spacing = (', pixel_spacing_x, ";", pixel_spacing_y, ";", pixel_spacing_z, ")")

    coord_iso_dose = (round((iso_x-position_image_x)/pixel_spacing_x),
                      round((iso_y-position_image_y)/pixel_spacing_y),
                      round((iso_z-position_image_z)/pixel_spacing_z))

    if verbose: print ('  > isocenter coodinates in dose matrix =', coord_iso_dose)

    doseimg = ds_dose.pixel_array[:,coord_iso_dose[1]+offset,:] * ds_dose.DoseGridScaling * 100 #en cGy

    if verbose: print ('  > maximum dose :', np.amax(doseimg))

    return (doseimg, dim_dose[0], dim_dose[2], pixel_spacing_x, pixel_spacing_z)




# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)

    
    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)
    

# IMG READING & REGISTRATION

In [37]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers



# Gafchromic Files:
m_path = r"\\interne.o-lambret.fr\oscar\RP\Commun\PHYSICIENS\Erwann\EBT3\14 - etalonnage lot 02282002\starHalcy"
m_nbOfFiles = 5
m_firstNb = 1
m_GafFilesName = r"\19H\pos03_0"
m_fileExtension = ".tif"

m_firstLine = 80  # Lines to be considered for processing
m_lastLine = 340 # Lines to be considered for processing
m_rectFilm = [40, 250, 115, 300]  # For the registration, all values outside this region will be set to 0

m_fillValue = 1.4   # value set outside the film for registration process

# Calculated Dose:
m_doseFileName = "\starHalcy1mm\RD1.dcm"
m_planFileName =  "\starHalcy1mm\RP1.dcm"

m_rectDose = [140, 340, 160, 340]   # [y1, y2, x1, x2]

dsd = 1000    # to compensate for the problems in field size
m_doseOffset = 0   # Offset to the central dose (-7 for lot 13)
m_doseFactor = (dsd-m_doseOffset)*(dsd-m_doseOffset)/(dsd*dsd)   # inverse square law


# Other
m_splineFile = "G://Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_normalCalib.txt"

m_dimViewer = 600




In [38]:
# READS THE FILES:


visu = True

# Reads the dose img:
(doseimg, dimx, dimy, pixsizex, pixsizey) = readDoseMatrix_AxialPlane_Isocenter(
    m_path+m_doseFileName, m_path+m_planFileName, offset=m_doseOffset, verbose=False)

calcDose = doseimg[m_rectDose[0]:m_rectDose[1], m_rectDose[2]:m_rectDose[3]]
# calcDose = np.flip(calcDose) * m_doseFactor
# calcDose = np.rot90(calcDose, 3) * m_doseFactor
calcDose = calcDose * m_doseFactor

# Reads and convert to dose the gafchromic film:
if m_nbOfFiles == 1:
    img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb)+m_fileExtension)
    rgbArr = sitk.GetArrayFromImage(img)
else:
    img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb)+m_fileExtension)
    size = (sitk.GetArrayFromImage(img).shape[0],
            sitk.GetArrayFromImage(img).shape[1],
            sitk.GetArrayFromImage(img).shape[2],
            m_nbOfFiles)
    imgs = np.zeros(size)
    for i in range(m_nbOfFiles):
        img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb+i)+m_fileExtension)
        imgs[:,:,:,i] = sitk.GetArrayFromImage(img)
    rgbArr = np.mean(imgs, axis=3)
    
rgbArr = subSampleRGBArray(rgbArr, 10)[m_firstLine:m_lastLine,:,:]
rbArr = rgbArr[:,:,0]/rgbArr[:,:,2]

size = (rbArr.shape[0],
        rbArr.shape[1])
regImg = np.zeros(size)
regImg.fill(m_fillValue)
regImg[m_rectFilm[0]:m_rectFilm[1], m_rectFilm[2]:m_rectFilm[3]] = \
        rbArr[m_rectFilm[0]:m_rectFilm[1], m_rectFilm[2]:m_rectFilm[3]]


try:
    g = Radiochromic_RB(m_path+m_GafFilesName, m_firstNb, m_nbOfFiles)
    g.subSampleDataArray(10)
    gafdoseimg = (g.convertToDose_cubicSplineFit(m_splineFile, 1000))[m_firstLine:m_lastLine,:]
except ValueError as err:
    print('Erreur: ' + err)



print('   >>> iMAGE rEAD !')

# shows dose images:
if visu:
    p1 = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer), 
               title='Calculated dose', toolbar_location="above")
    p1.image(image=[calcDose], x=0, y=0, dw=calcDose.shape[1], dh=calcDose.shape[1], palette="Plasma256")

    p2 = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer*rbArr.shape[0]/rbArr.shape[1]), 
               title='R/B image', toolbar_location="above")
    p2.image(image=[regImg], x=0, y=0, dw=rbArr.shape[1], dh=rbArr.shape[0], palette="Plasma256")

    show(p1)
    show(p2)


   >>> iMAGE rEAD !


In [39]:
# RIGID REGISTRATION USING SITK:

# Pour que la registration fonctionne, il est preferable de faire les translations pour commencer avec 
# initial transform (translation) et mettre l'image de dose dans le bon sens avec rotations / flip
# Une fois que c'est fait, il faut faire la final transform.


# Assigns images:
doseimg1 = regImg
doseimg2 = calcDose


# Initial transform:
fixedimg = sitk.Image(doseimg1.shape[1], doseimg1.shape[0], sitk.sitkFloat32)
for i in range(doseimg1.shape[1]):
    for j in range(doseimg1.shape[0]):
        fixedimg.SetPixel(i, j, float(doseimg1[j,i]))

movingimg = sitk.Image(doseimg2.shape[1], doseimg2.shape[0], sitk.sitkFloat32)
for i in range(doseimg2.shape[1]):
    for j in range(doseimg2.shape[0]):
        movingimg.SetPixel(i, j, float(doseimg2[j,i]))

        
# initial_transform = sitk.CenteredTransformInitializer(fixedimg, 
#                                                       movingimg, 
#                                                       sitk.Euler2DTransform(), 
#                                                       sitk.CenteredTransformInitializerFilter.GEOMETRY)
initial_transform = sitk.TranslationTransform(fixedimg.GetDimension(), (-120,-50))
# initial_transform = sitk.TranslationTransform(fixedimg.GetDimension(), (-50,-50))

# Real registration:
registration_method = sitk.ImageRegistrationMethod()


# Similarity metric settings:
#registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=20)
#registration_method.SetMetricAsJointHistogramMutualInformation(numberOfHistogramBins=5)
# registration_method.SetMetricAsMeanSquares()
registration_method.SetMetricAsCorrelation()

registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.1)

registration_method.SetInterpolator(sitk.sitkLinear)
# registration_method.SetInterpolator(sitk.sitkBSplineResamplerOrder5)


# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100000, 
                                                  convergenceMinimumValue=1e-9, convergenceWindowSize=10)
# registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=1.0, minStep=0.01, 
#                                                              numberOfIterations=100, relaxationFactor=0.5)
# registration_method.SetOptimizerAsRegularStepGradientDescent(learningRate=2.0, minStep=0.01, 
#                                                              numberOfIterations=2000, relaxationFactor=0.5)
registration_method.SetOptimizerScalesFromPhysicalShift()

registration_method.SetInitialTransform(initial_transform, inPlace=True)

final_transform = registration_method.Execute(sitk.Cast(fixedimg, sitk.sitkFloat32), 
                                              sitk.Cast(movingimg, sitk.sitkFloat32))
#itkfinalimg = sitk.Resample(movingimg, fixedimg, final_transform, sitk.sitkBSplineResamplerOrder5, 
#                         0.0, movingimg.GetPixelID())
itkfinalimg = sitk.Resample(movingimg, fixedimg, initial_transform, sitk.sitkBSplineResamplerOrder5, 
                         0.0, movingimg.GetPixelID())
finalimg = sitk.GetArrayFromImage(itkfinalimg)


print('   >>> rEGISTRATION dONE!')

   >>> rEGISTRATION dONE!


In [41]:
# COMPARE REGISTERED IMAGES:
# arr1 = np.nan_to_num(rbArr)
# arr1 = rbArr
# arr1 = regImg
arr1 = gafdoseimg

arr2 = finalimg
# max2 = np.amax(arr2)
# arr2 = arr2/max2

compare2Imgs(arr1, arr2, 220, 150, 
             title1="film", title2='dose')

# SAVING DATA ARRAY:

In [42]:
np.save("AI_files/lot02282002_HalcyonStar-Right_RGBArray.npy", rgbArr)
np.save("AI_files/lot02282002_HalcyonStar-Right_regDose.npy", finalimg)

In [ ]:
o = np.load("rgbArray.npy")
o.shape